### * RNN 주요 레이어 종류
#### (1) SimpleRNN :가장 간단한 형태의 RNN레이어, 활성화 함수로 tanh가 사용됨(tanh: -1 ~ 1 사이의 값을 반환)
#### (2) LSTM(Long short Term Memory) : 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제), LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함, 활성화 함수로 tanh외에 sigmoid가 사용됨
#### (3) GRU(Gated Recurent Unit) : 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수함

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
# Sequence data
X = np.array([[0,1,2,3],
              [1,2,3,4],
              [2,3,4,5],
              [3,4,5,6],
              [4,5,6,7],
              [5,6,7,8]],dtype=np.float32)

x_data = tf.reshape(X,(-1,4,1))  # (6,4,1)

y_data = np.array([4,5,6,7,8,9],dtype=np.float32)

print(x_data.shape,y_data.shape)
# print(type(x_data),type(y_data))
x_data

(6, 4, 1) (6,)


<tf.Tensor: shape=(6, 4, 1), dtype=float32, numpy=
array([[[0.],
        [1.],
        [2.],
        [3.]],

       [[1.],
        [2.],
        [3.],
        [4.]],

       [[2.],
        [3.],
        [4.],
        [5.]],

       [[3.],
        [4.],
        [5.],
        [6.]],

       [[4.],
        [5.],
        [6.],
        [7.]],

       [[5.],
        [6.],
        [7.],
        [8.]]], dtype=float32)>

## [1]SimpleRNN
### 가장 간단한 형태의 RNN

In [9]:
# RNN 순환 신경망 구현 : SimpleRNN
model = tf.keras.Sequential([
    # X: (N,D) , Wx: (D,H), Wh( (H,H), b: H
    #    (6,1),      (1,300)      (300,300)     300 --> 1*300+300*300+300 = 1500개의 Param
    # (N,T,D) : (6,4,1,) ---> (N,T,H) : (6,4,300) , T는 sequence_Length, H는 Hidden Size
    # return_sequence=True는 3차원(N,T,H) 으로 출력
    # return_sequence=False는 2차원(N,H) 으로 출력, 기본 값
    tf.keras.layers.SimpleRNN(units=300, return_sequences=True,input_shape=([4,1])),
    tf.keras.layers.SimpleRNN(units=300),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_9 (SimpleRNN)    (None, 4, 3)              15        
                                                                 
 simple_rnn_10 (SimpleRNN)   (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_data,y_data,epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 40.6130
Epoch 2/10
1/1 [==============================] - 0s 11ms/step - loss: 40.5319
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 40.4530
Epoch 4/10
1/1 [==============================] - 0s 15ms/step - loss: 40.3730
Epoch 5/10
1/1 [==============================] - 0s 18ms/step - loss: 40.2945
Epoch 6/10
1/1 [==============================] - 0s 10ms/step - loss: 40.2171
Epoch 7/10
1/1 [==============================] - 0s 15ms/step - loss: 40.1397
Epoch 8/10
1/1 [==============================] - 0s 14ms/step - loss: 40.0617
Epoch 9/10
1/1 [==============================] - 0s 16ms/step - loss: 39.9844
Epoch 10/10
1/1 [==============================] - 0s 14ms/step - loss: 39.9044


In [12]:
# 학습되지 않은 입력 데이터에 대한 예측 결과
print(model.predict(np.array([[[6.],[7.],[8.],[9.]]])))
print(model.predict(np.array([[[-1.],[0.],[1.],[2.]]])))
# 평가
model.evaluate(x_data, y_data)

1/1 [==============================] - 0s 130ms/step
[[0.4281078]]
1/1 [==============================] - 0s 12ms/step
[[0.1431872]]
1/1 [==============================] - 0s 134ms/step - loss: 39.8276


39.82759475708008

### [2] LSTM(Long short Term Memory)
#### 입력 데이터와 출력 사이의 거리가 멀어질수로 연관 관계가 적어진다(Long Term Dependency,장기의존성 문제)
#### LSTM은 장기 의존성 문제를 해결하기 위해 출력값외에 셀상태(cell state)값을 출력함

In [13]:
# RNN 순환 신경망 구현 : LSTM
model = tf.keras.Sequential([
    # X: (N,D) , Wx: (D,4H), Wh( (H,4H), b: H
    #    (6,1),      (1,300*4)      (300,300*4)     300*4 --> 1*300*4+300*300*4+300*4 = 4*90600개의 Param
    # (N,T,D) : (6,4,1,) ---> (N,T,H) : (6,4,300) , T는 sequence_Length, H는 Hidden Size
    # return_sequence=True는 3차원(N,T,H) 으로 출력
    # return_sequence=False는 2차원(N,H) 으로 출력, 기본 값
    tf.keras.layers.LSTM(units=300, return_sequences=True,input_shape=([4,1])),
    tf.keras.layers.LSTM(units=300),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 300)            362400    
                                                                 
 lstm_1 (LSTM)               (None, 300)               721200    
                                                                 
 dense_3 (Dense)             (None, 1)                 301       
                                                                 
Total params: 1,083,901
Trainable params: 1,083,901
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(x_data,y_data,epochs=100,verbose=0)
print(model.predict(x_data))

1/1 [==============================] - 0s 11ms/step
[[3.703652 ]
 [5.0507503]
 [6.1996517]
 [7.18532  ]
 [8.029896 ]
 [8.74765  ]]


### [3] GRU(Gated Recurent Unit)
#### 뉴욕대 조경현 교수 등이 제안, LSTM보다 구조가 간단하고 성능이 우수

In [17]:
# RNN 순환 신경망 구현 : GRU
model = tf.keras.Sequential([
    tf.keras.layers.GRU(units=300, return_sequences=True,input_shape=([4,1])),
    tf.keras.layers.GRU(units=300),
    tf.keras.layers.Dense(1)
])


model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 300)            272700    
                                                                 
 gru_1 (GRU)                 (None, 300)               541800    
                                                                 
 dense_4 (Dense)             (None, 1)                 301       
                                                                 
Total params: 814,801
Trainable params: 814,801
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(x_data,y_data,epochs=100,verbose=0)
print(model.predict(x_data))

1/1 [==============================] - 0s 262ms/step
[[3.6354423]
 [5.0283647]
 [6.2181554]
 [7.212613 ]
 [8.031939 ]
 [8.701498 ]]
